## ReMECS -- Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 

## Using 1D-CNN + SGD + One pass + RPWE

## Modifications:
- New Feature Set for EDA and Resp Belt. 
- Feature normalization is done.
- RECS with other modalities (EDA + Resp Belt) is incorporated for Real-time Multimodal Emotion Classification System (ReMECS).
- Weighted Majority Vote is used for Decision Fusion. 


## Date: 19 Sept 2022 at 10:10 AM

In [1]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import tensorflow as tf
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from window_slider import Slider

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
# def calculate_entropy(list_values):
#     counter_values = Counter(list_values).most_common()
#     probabilities = [elem[1]/len(list_values) for elem in counter_values]
#     entropy=scipy.stats.entropy(probabilities)
#     return entropy

# def calculate_statistics(list_values):
#     n5 = np.nanpercentile(list_values, 5)
#     n25 = np.nanpercentile(list_values, 25)
#     n75 = np.nanpercentile(list_values, 75)
#     n95 = np.nanpercentile(list_values, 95)
#     median = np.nanpercentile(list_values, 50)
#     mean = np.nanmean(list_values)
#     std = np.nanstd(list_values)
#     var = np.nanvar(list_values)
#     rms = np.nanmean(np.sqrt(list_values**2))
#     return [n5, n25, n75, n95, median, mean, std, var, rms]

# def calculate_crossings(list_values):
#     zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
#     no_zero_crossings = len(zero_crossing_indices)
#     mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
#     no_mean_crossings = len(mean_crossing_indices)
#     return [no_zero_crossings, no_mean_crossings]

# def get_features(list_values):    
#     list_values = list_values[0,:]
#     entropy = calculate_entropy(list_values)
#     crossings = calculate_crossings(list_values)
#     statistics = calculate_statistics(list_values)
#     return [entropy] + crossings + statistics

In [3]:
# #======================================================
# # EDA Feature Extraction (Wavelet Features)
# #======================================================
# def extract_eda_features(raw_eda):
#     features =[]
#     EDA = raw_eda
#     list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
# #     print(list_coeff)
#     for coeff in list_coeff:
#         features += get_features(coeff)
#     return features


In [4]:
# #======================================================
# # RESP BELT Feature Extraction (Wavelet Features)
# #======================================================

# def extract_resp_belt_features(raw_data):
#     features =[]
#     resp_belt = raw_data
#     list_coeff = pywt.wavedec(resp_belt, 'db4', level=3)
    
# #     print(list_coeff)
#     for coeff in list_coeff:
#         features += get_features(coeff)
#     return features

In [5]:
# def eeg_features(raw_data):
#     ch = 0
#     features= []
#     def calculate_entropy(list_values):
#         counter_values = Counter(list_values).most_common()
#         probabilities = [elem[1]/len(list_values) for elem in counter_values]
#         entropy=scipy.stats.entropy(probabilities)
#         return entropy

#     def calculate_statistics(list_values):
#         median = np.nanpercentile(list_values, 50)
#         mean = np.nanmean(list_values)
#         std = np.nanstd(list_values)
#         var = np.nanvar(list_values)
#         rms = np.nanmean(np.sqrt(list_values**2))
#         return [median, mean, std, var, rms]

#     def get_features(list_values):    
#     #     list_values = list_values[0,:]
#         entropy = calculate_entropy(list_values)
#         statistics = calculate_statistics(list_values)
#         return [entropy] + statistics
    
#     for i in range(raw_data.shape[0]):
#         ch_data = raw_data[i]
#         list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
#         for coeff in list_coeff:
#             features += get_features(coeff)
            
#         ch = ch+1
#     return features

In [8]:
#==================================
# 1D-CNN model
#==================================
def create_model(x):
    model = Sequential()
    model.add(Conv1D(filters=1, kernel_size=1, activation='relu', input_shape=(1,x.shape[1])))
    model.add(Conv1D(filters=1, kernel_size=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    opt = tf.keras.optimizers.SGD(learning_rate=0.05)


    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    
    print(model.summary())
    
    return model

In [9]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBbest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

beta_val = np.ones(num_classifiers)/num_classifiers
beta_aro = np.ones(num_classifiers)/num_classifiers


# optimizer= 'SGD' #optimizer
# classifier = 'MLP_RPWE'+str(optimizer)
indx = 0
c =0
ccc = 0

participant = 40
videos = 16

global eeg_emotion, eda_emotion, ecg_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
ecg_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()


eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()


ecg_acc_val = metrics.Accuracy() #Accuracy
ecg_f1m_val = metrics.F1() #F1 measure  
ecg_acc_aro = metrics.Accuracy() #Accuracy
ecg_f1m_aro = metrics.F1() #F1 measure
ecg_roc_val = metrics.ROCAUC()
ecg_roc_aro = metrics.ROCAUC()
ecg_mcc_val = metrics.MCC()
ecg_mcc_aro = metrics.MCC()

val_MSE = []
aro_MSE = []

haha_val = []
haha_aro = []

all_weights_aro = []
all_weights_val = []


 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
ecg_cm_val  = metrics.ConfusionMatrix()
ecg_cm_aro  = metrics.ConfusionMatrix()


    
itr = 0 #controls the learning rate


#=======================================
# MAIN Loop STARTS HERE
#=======================================
for ii in range(0,participant):

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = '/home/gp/Desktop/PhD-codes/AMIGOS_data/eeg_data/'+str(p)+'data_eeg_AMIGOS'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = '/home/gp/Desktop/PhD-codes/AMIGOS_data/eda_data/'+str(p)+'data_eda_AMIGOS.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # ECG data read from files
        ##===================================================
        def ecg_data(p):
            file_ecg = '/home/gp/Desktop/PhD-codes/AMIGOS_data/ecg_data/'+str(p)+'data_ecg_AMIGOS.csv'
            print(file_ecg)
            ecg_sig = pd.read_csv(file_ecg,sep=',', header = None, engine='python')
            return ecg_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_ecg = ecg_data(p)

        
        for jj in range(0,videos):
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            ecg_sig = grand_ecg.loc[grand_ecg.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            aro = eeg_sig.iloc[0,7087] #arousal 
            val = eeg_sig.iloc[0,7088] #valence
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al]) 
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            ecg_val_prdt=[]
            ecg_aro_prdt =[]
            
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,14),range(2,7087)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(2,7087)]) #keeping only eda signals
            ecg_sig = np.array(ecg_sig.iloc[range(0,2),range(2,7087)]) #keeping only ecg signals
            
            x_eeg = []
            x_ecg = []
            
            x_eda = eda_sig
            
            for rr in range(0,eeg_sig.shape[0]):
                for cc in range(0,eeg_sig.shape[1]):
                    x_eeg.append(eeg_sig[rr,cc])
                    
            for rr in range(0,ecg_sig.shape[0]):
                for cc in range(0,ecg_sig.shape[1]):
                    x_ecg.append(ecg_sig[rr,cc])
                    
            x_eeg = np.array([x_eeg])
            x_ecg = np.array([x_ecg])
            
            
            #===================================================
            # Model initialization
            #===================================================
            if init_m == 0:
                print('EEG Feature shape{}:'.format(x_eeg.shape))
                print('EDA Feature shape{}:'.format(x_eda.shape))
                print('ecg BELT Feature shape{}:'.format(x_ecg.shape))

                #========================
                # For EEG data model
                #========================
                eeg_model_val = create_model(x_eeg)

                eeg_model_aro = create_model(x_eeg)

                #========================
                # For EDA data model
                #========================
                eda_model_val = create_model(x_eda)

                eda_model_aro = create_model(x_eda)

                #==============================
                # For ecg Belt data Model
                #==============================
                ecg_model_val = create_model(x_ecg)

                ecg_model_aro = create_model(x_ecg)

                init_m = init_m+1


            #===============================================================
            # Emotion Classification --> Valence and Arousal
            #===============================================================

            if c == 0: #For the first time model will return 0 or None
                tmp_eeg_val = 0
                tmp_eeg_aro = 0

                tmp_eda_val = 0
                tmp_eda_aro = 0

                tmp_ecg_val = 0
                tmp_ecg_aro = 0


                y_pred_val_eeg = [2]
                y_pred_aro_eeg = [2]

                y_pred_val_eda = [2]
                y_pred_aro_eda = [2]

                y_pred_val_ecg = [2]
                y_pred_aro_ecg = [2]

                eeg_model_val.fit(np.array([x_eeg]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)

                eda_model_val.fit(np.array([x_eda]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)

                ecg_model_val.fit(np.array([x_ecg]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                ecg_model_aro.fit(np.array([x_ecg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)

                c=c+1

            else: 

                tmp_eeg_val  = eeg_model_val.predict(np.array([x_eeg]))[0][0]
                tmp_eeg_aro = eeg_model_aro.predict(np.array([x_eeg]))[0][0]

                tmp_eda_val = eda_model_val.predict(np.array([x_eda]))[0][0]
                tmp_eda_aro  = eda_model_aro.predict(np.array([x_eda]))[0][0]

                tmp_ecg_val = ecg_model_val.predict(np.array([x_ecg]))[0][0]
                tmp_ecg_aro = ecg_model_aro.predict(np.array([x_ecg]))[0][0]

                #Binary output scaling 1 if x > 0.5 else 0
                y_pred_val_eeg = [1 if tmp_eeg_val > 0.5 else 0]
                y_pred_aro_eeg = [1 if tmp_eeg_aro > 0.5 else 0]

                y_pred_val_eda = [1 if tmp_eda_val > 0.5 else 0]
                y_pred_aro_eda = [1 if tmp_eda_aro > 0.5 else 0]

                y_pred_val_ecg = [1 if tmp_ecg_val > 0.5 else 0]
                y_pred_aro_ecg = [1 if tmp_ecg_aro > 0.5 else 0]

                #Fit the model once
                eeg_model_val.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)

                eda_model_val.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)

                ecg_model_val.fit(np.array([x_ecg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                ecg_model_aro.fit(np.array([x_ecg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)



            #===========================================
            # Performance matric update
            #===========================================

            #Valence classification EEG

            eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg[0])  # update the accuracy metric

            eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg[0]) #update f1 measure metric

            eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg[0])

            eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg[0])

            #Arousal classification EEG 

            eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])  # update the accuracy metric

            eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg[0]) #update f1 measure metric

            eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])

            eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])


            #Valence classification EDA

            eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda[0])  # update the accuracy metric

            eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda[0]) #update f1 measure metric

            eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda[0])

            eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda[0])


            #Arousal classification EDA

            eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda[0])  # update the accuracy metric

            eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda[0]) #update f1 measure metric

            eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda[0])

            eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda[0])

            #Valence classification ecg Belt

            ecg_acc_val = ecg_acc_val.update(y_act_val[0], y_pred_val_ecg[0])  # update the accuracy metric

            ecg_f1m_val = ecg_f1m_val.update(y_act_val[0], y_pred_val_ecg[0]) #update f1 measure metric

            ecg_roc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg[0])

            ecg_mcc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg[0])


            #Arousal classification ecg Belt

            ecg_acc_aro = ecg_acc_aro.update(y_act_aro[0], y_pred_aro_ecg[0])  # update the accuracy metric

            ecg_f1m_aro = ecg_f1m_aro.update(y_act_aro[0], y_pred_aro_ecg[0]) #update f1 measure metric

            ecg_roc_aro = ecg_roc_aro.update(y_act_aro[0], y_pred_aro_ecg[0])

            ecg_mcc_aro = ecg_mcc_aro.update(y_act_aro[0], y_pred_aro_ecg[0])

                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))
            
            ecg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_ecg_val))
            ecg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_ecg_aro))
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, ecg_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, ecg_aro_MSE] #Arousal errors
            

            
            #=================================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #=================================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
                
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
            
            #============================================
            # Controls for the first time
            #============================================
            if ccc ==0:
                val_label = [2, 2, 2]
                aro_label = [2, 2, 2] 
                ccc =ccc+1
            else:
                val_label = [tmp_eeg_val, tmp_eda_val, tmp_ecg_val]
                aro_label = [tmp_eeg_aro, tmp_eda_aro, tmp_ecg_aro] 
            
            
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg[0])
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda[0])
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda[0])
            
            ecg_cm_val = ecg_cm_val.update(y_act_val[0], y_pred_val_ecg[0])
            ecg_cm_varo = ecg_cm_aro.update(y_act_aro[0], y_pred_aro_ecg[0])            
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot(val_label, w_val)
            mer_val = 1 if p_val > 0.5 else 0
            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_aro = np.dot(aro_label,w_aro)
            mer_aro = 1 if p_aro > 0.5 else 0
            
#             print([val_label, aro_label])
#             print([p_val,p_aro])
            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling
            
            #============================================
            # Weight update for Arousal
            #============================================
            
            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            all_weights_val.append(w_val)
            all_weights_aro.append(w_aro)
            
#             mer_val = val_label[np.argmax(w_val)] #Final Class prediction Valence
#             mer_aro = aro_label[np.argmax(w_aro)] #Final Class Prediction Valence
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val  = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro  = mer_cm_aro.update(y_act_aro[0], mer_aro) 
            
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                        eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                        y_act_val[0], y_pred_val_eeg[0], y_act_aro[0], y_pred_aro_eeg[0]]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                        eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                        y_act_val[0], y_pred_val_eda[0], y_act_aro[0], y_pred_aro_eda[0]]))
            
            ecg_emotion.append(np.array([p,v, ecg_acc_val.get(), ecg_f1m_val.get(), ecg_roc_val.get(), ecg_mcc_val.get(),
                                          ecg_acc_aro.get(),ecg_f1m_aro.get(), ecg_roc_aro.get(), ecg_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_ecg[0], y_act_aro[0], y_pred_aro_ecg[0]]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                        mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                        y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/PhD-codes/AMIGOS_data/eeg_data/1data_eeg_AMIGOS.csv
/home/gp/Desktop/PhD-codes/AMIGOS_data/eda_data/1data_eda_AMIGOS.csv
/home/gp/Desktop/PhD-codes/AMIGOS_data/ecg_data/1data_ecg_AMIGOS.csv
Person: 1 Video:1
EEG Feature shape(1, 99190):
EDA Feature shape(1, 7085):
ecg BELT Feature shape(1, 14170):
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1, 1)              99191     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 1)              2         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1)              0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (

MER model
Valence Accuracy:0.0
Valence F1 score:0.0
Valence ROC score:-0.0
Valence MCC score:0.0
Arousal Accuracy:0.0
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:2
MER model
Valence Accuracy:0.0
Valence F1 score:0.0
Valence ROC score:-0.0
Valence MCC score:-1.0
Arousal Accuracy:0.5
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:3
MER model
Valence Accuracy:0.0
Valence F1 score:0.0
Valence ROC score:-0.0
Valence MCC score:-1.0
Arousal Accuracy:0.6667
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:4
MER model
Valence Accuracy:0.0
Valence F1 score:0.0
Valence ROC score:-0.0
Valence MCC score:-1.0
Arousal Accuracy:0.75
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0

MER model
Valence Accuracy:0.3125
Valence F1 score:0.0
Valence ROC score:0.3125
Valence MCC score:-0.4804
Arousal Accuracy:0.5625
Arousal F1 score:0.0
Arousal ROC score:0.45
Arousal MCC score:-0.2
===============================xxxx================================
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/PhD-codes/AMIGOS_data/eeg_data/2data_eeg_AMIGOS.csv
/home/gp/Desktop/PhD-codes/AMIGOS_data/eda_data/2data_eda_AMIGOS.csv
/home/gp/Desktop/PhD-codes/AMIGOS_data/ecg_data/2data_ecg_AMIGOS.csv
Person: 2 Video:1
MER model
Valence Accuracy:0.3529
Valence F1 score:0.1538
Valence ROC score:0.3681
Valence MCC score:-0.3105
Arousal Accuracy:0.5882
Arousal F1 score:0.0
Arousal ROC score:0.4545
Arousal MCC score:-0.1846
===============================xxxx================================
Person: 2 Video:2
MER model
Valence Accuracy:0.3889
Valence F1 score:0.2667
Valence ROC score:0.4125
Valence MCC score:-0.1941
Arousa

MER model
Valence Accuracy:0.3514
Valence F1 score:0.2
Valence ROC score:0.3465
Valence MCC score:-0.3278
Arousal Accuracy:0.6757
Arousal F1 score:0.0
Arousal ROC score:0.4808
Arousal MCC score:-0.1084
===============================xxxx================================
Person: 3 Video:6
MER model
Valence Accuracy:0.3684
Valence F1 score:0.2
Valence ROC score:0.3583
Valence MCC score:-0.3043
Arousal Accuracy:0.6579
Arousal F1 score:0.0
Arousal ROC score:0.4808
Arousal MCC score:-0.1117
===============================xxxx================================
Person: 3 Video:7
MER model
Valence Accuracy:0.3846
Valence F1 score:0.2
Valence ROC score:0.369
Valence MCC score:-0.2829
Arousal Accuracy:0.6667
Arousal F1 score:0.0
Arousal ROC score:0.4815
Arousal MCC score:-0.1081
===============================xxxx================================
Person: 3 Video:8
MER model
Valence Accuracy:0.4
Valence F1 score:0.2
Valence ROC score:0.3788
Valence MCC score:-0.2632
Arousal Accuracy:0.675
Arousal F1 

MER model
Valence Accuracy:0.4746
Valence F1 score:0.1622
Valence ROC score:0.4276
Valence MCC score:-0.1776
Arousal Accuracy:0.5593
Arousal F1 score:0.0
Arousal ROC score:0.4853
Arousal MCC score:-0.1126
===============================xxxx================================
Person: 4 Video:12
MER model
Valence Accuracy:0.4667
Valence F1 score:0.1579
Valence ROC score:0.4253
Valence MCC score:-0.185
Arousal Accuracy:0.55
Arousal F1 score:0.0
Arousal ROC score:0.4853
Arousal MCC score:-0.1138
===============================xxxx================================
Person: 4 Video:13
MER model
Valence Accuracy:0.459
Valence F1 score:0.1538
Valence ROC score:0.4232
Valence MCC score:-0.1919
Arousal Accuracy:0.541
Arousal F1 score:0.0
Arousal ROC score:0.4853
Arousal MCC score:-0.115
===============================xxxx================================
Person: 4 Video:14
MER model
Valence Accuracy:0.4516
Valence F1 score:0.15
Valence ROC score:0.4212
Valence MCC score:-0.1985
Arousal Accuracy:0.5484

/home/gp/Desktop/PhD-codes/AMIGOS_data/eda_data/6data_eda_AMIGOS.csv
/home/gp/Desktop/PhD-codes/AMIGOS_data/ecg_data/6data_ecg_AMIGOS.csv
Person: 6 Video:1
MER model
Valence Accuracy:0.4444
Valence F1 score:0.1176
Valence ROC score:0.4155
Valence MCC score:-0.2225
Arousal Accuracy:0.6049
Arousal F1 score:0.4839
Arousal ROC score:0.6389
Arousal MCC score:0.339
===============================xxxx================================
Person: 6 Video:2
MER model
Valence Accuracy:0.439
Valence F1 score:0.1154
Valence ROC score:0.4145
Valence MCC score:-0.2267
Arousal Accuracy:0.5976
Arousal F1 score:0.4762
Arousal ROC score:0.6261
Arousal MCC score:0.3033
===============================xxxx================================
Person: 6 Video:3
MER model
Valence Accuracy:0.4458
Valence F1 score:0.1154
Valence ROC score:0.4173
Valence MCC score:-0.2202
Arousal Accuracy:0.6024
Arousal F1 score:0.4923
Arousal ROC score:0.6334
Arousal MCC score:0.3156
===============================xxxx==================

MER model
Valence Accuracy:0.4608
Valence F1 score:0.0984
Valence ROC score:0.4319
Valence MCC score:-0.1972
Arousal Accuracy:0.6176
Arousal F1 score:0.5979
Arousal ROC score:0.6411
Arousal MCC score:0.2883
===============================xxxx================================
Person: 7 Video:7
MER model
Valence Accuracy:0.466
Valence F1 score:0.0984
Valence ROC score:0.4337
Valence MCC score:-0.1927
Arousal Accuracy:0.6117
Arousal F1 score:0.5918
Arousal ROC score:0.6321
Arousal MCC score:0.2695
===============================xxxx================================
Person: 7 Video:8
MER model
Valence Accuracy:0.4615
Valence F1 score:0.0968
Valence ROC score:0.433
Valence MCC score:-0.1956
Arousal Accuracy:0.6154
Arousal F1 score:0.6
Arousal ROC score:0.6364
Arousal MCC score:0.277
===============================xxxx================================
Person: 7 Video:9
MER model
Valence Accuracy:0.4667
Valence F1 score:0.0968
Valence ROC score:0.4348
Valence MCC score:-0.1912
Arousal Accuracy:0

MER model
Valence Accuracy:0.4677
Valence F1 score:0.1538
Valence ROC score:0.4407
Valence MCC score:-0.1576
Arousal Accuracy:0.6452
Arousal F1 score:0.6765
Arousal ROC score:0.6568
Arousal MCC score:0.3057
===============================xxxx================================
Person: 8 Video:13
MER model
Valence Accuracy:0.464
Valence F1 score:0.1519
Valence ROC score:0.4398
Valence MCC score:-0.1606
Arousal Accuracy:0.64
Arousal F1 score:0.6715
Arousal ROC score:0.6496
Arousal MCC score:0.2922
===============================xxxx================================
Person: 8 Video:14
MER model
Valence Accuracy:0.4683
Valence F1 score:0.1519
Valence ROC score:0.4414
Valence MCC score:-0.1567
Arousal Accuracy:0.6349
Arousal F1 score:0.6667
Arousal ROC score:0.6426
Arousal MCC score:0.2792
===============================xxxx================================
Person: 8 Video:15
MER model
Valence Accuracy:0.4646
Valence F1 score:0.15
Valence ROC score:0.4406
Valence MCC score:-0.1596
Arousal Accura

Person: 10 Video:1
MER model
Valence Accuracy:0.469
Valence F1 score:0.1348
Valence ROC score:0.4467
Valence MCC score:-0.1511
Arousal Accuracy:0.6621
Arousal F1 score:0.6879
Arousal ROC score:0.6676
Arousal MCC score:0.3303
===============================xxxx================================
Person: 10 Video:2
MER model
Valence Accuracy:0.4658
Valence F1 score:0.1333
Valence ROC score:0.4461
Valence MCC score:-0.1534
Arousal Accuracy:0.6575
Arousal F1 score:0.6835
Arousal ROC score:0.6619
Arousal MCC score:0.3194
===============================xxxx================================
Person: 10 Video:3
MER model
Valence Accuracy:0.4694
Valence F1 score:0.1333
Valence ROC score:0.4473
Valence MCC score:-0.1503
Arousal Accuracy:0.6531
Arousal F1 score:0.6792
Arousal ROC score:0.6564
Arousal MCC score:0.3089
===============================xxxx================================
Person: 10 Video:4
MER model
Valence Accuracy:0.4662
Valence F1 score:0.1319
Valence ROC score:0.4467
Valence MCC score

KeyboardInterrupt: 

In [17]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('RESP Val CM:')
print(resp_cm_val)
print('RESP Aro CM:')
print(resp_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    297    275
      1    164    544
EEG aro CM:
             0      1
      0    140    403
      1    318    419
EDA Val CM:
             0      1
      0    295    277
      1    162    546
EDA aro CM:
             0      1
      0    139    404
      1    318    419
RESP Val CM:
             0      1
      0    296    276
      1    162    546
RESP Aro CM:
             0      1
      0    139    404
      1    318    419
ReMECS Val CM:
             0      1
      0    296    276
      1    163    545
ReMECS aro CM:
             0      1
      0    139    404
      1    318    419


In [ ]:
print('[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]')
#=============================
# EEG
#=============================
t_eeg_emotion =pd.DataFrame(eeg_emotion) 
t_eeg_emotion = t_eeg_emotion.iloc[:,range(2,10)]
t_eeg_emotion_avg = np.array(np.mean(t_eeg_emotion))
t_eeg_emotion_std = np.array(np.std(t_eeg_emotion))

print('EEG result Average:', t_eeg_emotion_avg)
print('EEG result std:', t_eeg_emotion_std)

#=============================
# EDA
#=============================
t_eda_emotion =pd.DataFrame(eda_emotion) 
t_eda_emotion = t_eda_emotion.iloc[:,range(2,10)]
t_eda_emotion_avg = np.array(np.mean(t_eda_emotion))
t_eda_emotion_std = np.array(np.std(t_eda_emotion))

print('EDA result Average:', t_eda_emotion_avg)
print('EDA result std:', t_eda_emotion_std)
#=============================
# RESP BELT
#=============================
t_resp_emotion =pd.DataFrame(resp_emotion) 
t_resp_emotion = t_resp_emotion.iloc[:,range(2,10)]
t_resp_emotion_avg = np.array(np.mean(t_resp_emotion))
t_resp_emotion_std = np.array(np.std(t_resp_emotion))

print('Resp Belt result Average:', t_resp_emotion_avg)
print('Resp Belt result std:', t_resp_emotion_std)
#=============================
# MER
#=============================
t_mer_emotion =pd.DataFrame(mer_emotion) 
t_mer_emotion = t_mer_emotion.iloc[:,range(2,10)]
t_mer_emotion_avg = np.array(np.mean(t_mer_emotion))
t_mer_emotion_std = np.array(np.std(t_mer_emotion))

print('MER result Average:', t_mer_emotion_avg)
print('MER result std:', t_mer_emotion_std)

In [18]:
classifier = '1D_CNN_'+str(optimizer)

In [19]:
time_stamp = datetime.datetime.now()
run =1
algo= '1DCNN/' 
filename = "/home/gp/Desktop/MER_arin/12DEAP_RPWE/Results_RPWE_DEAP/"+algo
fname_eeg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EDA'+'_' +classifier+'_results.csv'
fname_resp = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_RESP_BELT'+'_' +classifier+'_results.csv'
fname_mer = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
resp_emotion = pd.DataFrame(resp_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
resp_emotion.to_csv(fname_resp)
mer_emotion.to_csv(fname_mer)


